In [504]:
%load_ext autoreload
%autoreload 2

from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import scikitplot as skplot\
    
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA
from pipelinehelper import PipelineHelper
from sklearn import preprocessing
from sklearn.metrics import make_scorer

from notebooks.context import ml_project
from ml_project.task_1.transformations import aggregate_feature_matrix, get_phi_callables
from ml_project.io import DataHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [505]:
# To surpress sklearn warnings
def warn(*args, **kwargs):
    pass
import warnings

warnings.warn = warn
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [81]:
DIR_NAME = '/Users/sluck/eth/introML/data/task1b'

In [506]:
from sklearn.neighbors import LocalOutlierFactor 
data_handler = DataHandler(DIR_NAME)
all_data = data_handler.load_train_data()

# 80 20 is standart?
X_train, X_test, y_train, y_test = train_test_split(all_data.drop(['y'], axis=1), 
                                                    all_data['y'], test_size=0.2,
                                                    random_state=131
                                                    )

# Apply feature transforms
feature_mat_all = pd.concat([aggregate_feature_matrix(all_data, get_phi_callables()), all_data['y']], axis=1)
feature_mat_train = aggregate_feature_matrix(X_train, get_phi_callables())
feature_mat_test = aggregate_feature_matrix(X_test, get_phi_callables())

#outlier detect?




Aggregated feature matrix of shape: (1000, 21)
Aggregated feature matrix of shape: (800, 21)
Aggregated feature matrix of shape: (200, 21)


In [507]:
import math
from sklearn.metrics import mean_squared_error

def my_scorer(y_true, y_predicted):
    error = mean_squared_error(y_true, y_predicted)**0.5
    return error

In [508]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression,VarianceThreshold
from sklearn.linear_model import ElasticNet,RidgeCV,Perceptron,TheilSenRegressor,LarsCV
from sklearn.linear_model import Lasso,LassoCV
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.ensemble import IsolationForest

from sklearn.cluster import DBSCAN


#print(feature_mat_train[['phi_1','phi_6','phi_11','phi_16']])
from sklearn.cluster import DBSCAN
outlier_detection = DBSCAN(
  eps = 3.6,#3.1 for entire
  metric="euclidean",
  min_samples = 50,
  n_jobs = -1)
clusters = outlier_detection.fit_predict(feature_mat_train)

#clusters = outlier_detection.fit_predict(
    #feature_mat_train[['phi_1','phi_2','phi_3','phi_4','phi_5']])
# remove outliers
#print(feature_mat_train)
print(feature_mat_train.shape,clusters.shape)
new_train=feature_mat_train
new_train['outlier']=clusters
feature_mat_train_new=new_train[new_train.outlier==0]
feature_mat_train_new=feature_mat_train_new.drop(['outlier'],axis=1)
#print(feature_mat_train_new.head())

print("Done train")

print(y_train.shape,clusters.shape)
new_y=y_train.to_frame()
print(new_y.shape,clusters.shape)
print(type(new_train),type(feature_mat_train_new))
new_y['outlier']=clusters
y_train_new=new_y[new_y.outlier==0]
y_train_new=y_train_new.drop(['outlier'],axis=1)

print(f"We are left with {y_train_new.shape}")




(800, 21) (800,)
Done train
(800,) (800,)
(800, 1) (800,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
We are left with (626, 1)


# Fitting the linear model and get weight vector

In [510]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
my_func = make_scorer(my_scorer,greater_is_better=False)
model=SelectKBest(k=13)
test_sel=model.fit(feature_mat_train_new,y_train_new)
test=test_sel.get_support(indices=True)
print(test)
print(test_sel.scores_)
# lets remove all labels below 16
# feature_mat_train_sel=feature_mat_train_new[
#     [f'phi_{i}' for i in range(16,21)]
# ]

for i in range(1,16):
    feature_mat_train_sel[f'phi_{i}']=0
feature_mat_train_sel['phi_21']=0



pipe = Pipeline([
                #('std_scale', StandardScaler()),
                #('preprocess',([]))
                #('minMaxScale',preprocessing.MinMaxScaler()),
                # ('scale', PipelineHelper([
                #      ('std_scale', StandardScaler()),
                #      ('minMaxScale',preprocessing.MinMaxScaler()),
                #   ])),
                #('feature_select', SelectKBest()),
                #('VR',VarianceThreshold()),
                 #('baye_ridge', BayesianRidge()),
                #('orthoMP',OrthogonalMatchingPursuitCV()),
                 #('Lasso',Lasso()),
                #('elas',ElasticNet()),
                  ('regr', PipelineHelper([
                      #('linregr', LinearRegression()),
                      ('ridge', Ridge()),
                      #('BR', BayesianRidge(fit_intercept=False)),
                      # ('RidgeCV',RidgeCV()),
                      # ('LarsCV',LarsCV()),
                       #('Lasso',Lasso()),
                      #('LassoCV',LassoCV(fit_intercept=False)),
                      # ('TheilSenRegressor',TheilSenRegressor()),
                      
                      #('ARDRegression',ARDRegression())
                  #     ('elas',ElasticNet()),
                     #('orthoMP',OrthogonalMatchingPursuitCV()),
                      #ortho doesnt work at all coefficients not good apperantly
                  ])),                   
])

param_grid = {
    #'feature_select__k': [3],
    #'VR__threshold':[1/(i*i*i) for i in range(1,10)],
    #'orthoMP__cv':[8,9,10,11],
    #'orthoMP__max_iter':[2,3,4,5],
    #'elas__max_iter':[100,1000],    


    'regr__selected_model': pipe.named_steps['regr'].generate({
        #'ridge__alpha': [0.01, 0.1, 1, 10, 100, 1000],
        #'orthoMP__max_iter':[2,3,4,5,7,9],
        # 'LassoCV__cv':[3,5,10],
        # 'LassoCV__n_alphas':[20],
        # 'LassoCV__positive':[False],
        # 'LassoCV__selection':['cyclic'],
        # 'LassoCV__tol':[1.0e-20],
        'ridge__alpha': [500],
        'ridge__fit_intercept':[False],
        #'BR__tol':[1.0e-20],
        #'BR__compute_score':[True],
        #'Lasso__alpha':[512],
        
    })
}
print("shape:",feature_mat_train_new.shape)
grid_cv = GridSearchCV(pipe, param_grid=param_grid,
                       cv=10, scoring=my_func, 
                       verbose=False, refit=True)

grid_cv = grid_cv.fit(feature_mat_train_new, y_train_new)


[ 8  9 10 11 12 13 14 15 16 17 18 19 20]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan]
shape: (626, 21)


In [511]:
from ml_project.train.gridcv import print_gridcv_report
print_gridcv_report(grid_cv, neg_sqr_of_score=True)

Best param set: 
{'regr__selected_model': ('ridge', {'alpha': 500, 'fit_intercept': False})}

 ----------
Mean test scores for parameter combinations...
3.165 (+/- 1.540) for {'regr__selected_model': ('ridge', {'alpha': 500, 'fit_intercept': False})}
----------


In [512]:
from sklearn.metrics import mean_squared_error

print(feature_mat_test.shape)
y_pred = grid_cv.predict(feature_mat_test)
print(y_pred.shape,y_test.shape)

print('RMSE on out of sample test set:', mean_squared_error(y_test, y_pred)**0.5)

(200, 21)
(200, 1) (200,)
RMSE on out of sample test set: 9.42841913367115


In [513]:
best_estimator = grid_cv.best_estimator_
regr_model = best_estimator.named_steps['regr'].selected_model
try:
    print(best_estimator.named_steps['VR'].get_support(indices=True))
except Exception as e:
    print(f"E> {e}")
    pass

coefficients = regr_model.coef_

E> 'VR'


In [514]:
def print_final_weights(coefficients):
    print('Coefficients (weights) for feature transforms...\n')
    for feat_name, coeff in zip(get_phi_callables().keys(), coefficients):
        print('\t{}: {}'.format(feat_name, coeff))
        
print_final_weights(coefficients)

Coefficients (weights) for feature transforms...

	phi_1: [ 0.83824709 -0.36363283 -0.19899983  0.19176308 -0.11030599 -0.8828808
  0.17577547 -0.62196221 -0.1289628   0.4156626   0.47954058 -0.59116605
 -0.92879815 -0.04192099 -0.15666939  0.13928855 -0.27241754 -0.0027968
 -0.15434063 -0.34294541 -0.2162026 ]


In [515]:
print('At all',coefficients)
print('At o',coefficients[0])
try:
    assert len(coefficients)==21,f'We need 21 in output but was {len(coefficients)}'
    data_handler.store_results_task1b(list(coefficients))
    print("Done")
except Exception as e:
    print(f"Diffrent format {e}")
    assert len(coefficients[0])==21,f'We need 21 in output but was {len(coefficients[0])}'
    data_handler.store_results_task1b(coefficients[0])
    print(coefficients[0])

At all [[ 0.83824709 -0.36363283 -0.19899983  0.19176308 -0.11030599 -0.8828808
   0.17577547 -0.62196221 -0.1289628   0.4156626   0.47954058 -0.59116605
  -0.92879815 -0.04192099 -0.15666939  0.13928855 -0.27241754 -0.0027968
  -0.15434063 -0.34294541 -0.2162026 ]]
At o [ 0.83824709 -0.36363283 -0.19899983  0.19176308 -0.11030599 -0.8828808
  0.17577547 -0.62196221 -0.1289628   0.4156626   0.47954058 -0.59116605
 -0.92879815 -0.04192099 -0.15666939  0.13928855 -0.27241754 -0.0027968
 -0.15434063 -0.34294541 -0.2162026 ]
Diffrent format We need 21 in output but was 1
[ 0.83824709 -0.36363283 -0.19899983  0.19176308 -0.11030599 -0.8828808
  0.17577547 -0.62196221 -0.1289628   0.4156626   0.47954058 -0.59116605
 -0.92879815 -0.04192099 -0.15666939  0.13928855 -0.27241754 -0.0027968
 -0.15434063 -0.34294541 -0.2162026 ]
